# Práctica 2 Opcional

### Aplicación de transformaciones geométricas básicas: Marcar el punto de giro y trasladar la imagen arrastrando con el ratón

In [ ]:
import cv2 as cv
import numpy as np

img = cv.imread('images/cats.png')
if img is None:
    raise FileNotFoundError("No se encontró 'images/cats.png'")
h, w = img.shape[:2]

# Lienzo 2x
CW, CH = w*2, h*2
ox, oy = (CW - w)//2, (CH - h)//2
canvas0 = np.zeros((CH, CW, 3), np.uint8)
canvas0[oy:oy+h, ox:ox+w] = img # imagen en el centro del lienzo

cv.namedWindow("Transformaciones", cv.WINDOW_AUTOSIZE)
def nothing(_): pass

cv.createTrackbar("Tx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Ty", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Cy", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 300, nothing)  # 1.00 = 100
cv.createTrackbar("EscalaY", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("Controles:")
print("- Clic izquierdo y arrastrar: trasladar imagen.")
print("- Clic derecho: fijar centro de rotación/escalado en el punto clicado.")
print("- ESC para salir.")

# Transformación afín:
#     T(x) = A·(x - c) + c + t
# donde:
#     A -> rotación y escala combinadas
#     c -> centro de rotación/escalado
#     t -> traslación global
#
# OpenCV aplica:  x' = A·x + b
# donde:
#     b = t + (I - A)·c

t_state = np.array([0.0, 0.0], dtype=np.float32) # vector de traslación global (t_x, t_y) que desplaza toda la imagen
prev_c = np.array([                              # centro anterior de rotación/escalado (c_x, c_y), usado para compensar movimiento
    cv.getTrackbarPos("Cx", "Transformaciones"),
    cv.getTrackbarPos("Cy", "Transformaciones")
], dtype=np.float32)
current_c = prev_c.copy()

# Variables globales para el callback del mouse
A_cur = np.eye(2, dtype=np.float32)
I_cur = np.eye(2, dtype=np.float32)
M_cur = np.hstack([A_cur, np.zeros((2,1), np.float32)])

dragging = False
drag_start_canvas = np.array([0.0, 0.0], dtype=np.float32)
drag_base_t = np.array([0.0, 0.0], dtype=np.float32)

def mouse(event, x, y, flags, param):
    global dragging, drag_start_canvas, drag_base_t
    global t_state, prev_c, current_c, A_cur, I_cur, M_cur

    gx, gy = x + ox, y + oy

    if event == cv.EVENT_LBUTTONDOWN:
        dragging = True
        drag_start_canvas = np.array([gx, gy], dtype=np.float32)
        drag_base_t = t_state.copy()

    elif event == cv.EVENT_MOUSEMOVE and dragging:
        delta = np.array([gx, gy], dtype=np.float32) - drag_start_canvas # desplazamiento desde el inicio del arrastre
        t_state = drag_base_t + delta

        # Reflejar cambios en los trackbars
        cv.setTrackbarPos("Tx", "Transformaciones", int(np.clip(round(t_state[0]) + CW//2, 0, CW)))
        cv.setTrackbarPos("Ty", "Transformaciones", int(np.clip(round(t_state[1]) + CH//2, 0, CH)))

    elif event == cv.EVENT_LBUTTONUP:
        dragging = False

    elif event == cv.EVENT_RBUTTONDOWN:
        # Compensar cambio de pivote manteniendo b
        M_inv = cv.invertAffineTransform(M_cur)
        cx_src = M_inv[0,0]*gx + M_inv[0,1]*gy + M_inv[0,2] # puntos en la imagen original
        cy_src = M_inv[1,0]*gx + M_inv[1,1]*gy + M_inv[1,2] # puntos en la imagen original
        c_new = np.array([cx_src, cy_src], dtype=np.float32)
        
        b_keep = t_state + (I_cur - A_cur) @ prev_c
        t_state = b_keep - (I_cur - A_cur) @ c_new

        cv.setTrackbarPos("Cx", "Transformaciones", int(np.clip(round(c_new[0]), 0, CW)))
        cv.setTrackbarPos("Cy", "Transformaciones", int(np.clip(round(c_new[1]), 0, CH)))
        cv.setTrackbarPos("Tx", "Transformaciones", int(np.clip(round(t_state[0]) + CW//2, 0, CW)))
        cv.setTrackbarPos("Ty", "Transformaciones", int(np.clip(round(t_state[1]) + CH//2, 0, CH)))

        prev_c = c_new.copy()
        current_c = c_new.copy()

cv.setMouseCallback("Transformaciones", mouse)

while True:
    tx_ui = cv.getTrackbarPos("Tx", "Transformaciones") - CW//2
    ty_ui = cv.getTrackbarPos("Ty", "Transformaciones") - CH//2
    ang = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    if cv.getTrackbarPos("Uniforme", "Transformaciones") == 1:
        sy = sx

    # Rotación y escalado
    rad = np.deg2rad(ang)
    co, si = np.cos(rad), np.sin(rad)
    R = np.array([[ co, -si],
                  [ si,  co]], dtype=np.float32)
    S = np.array([[sx, 0.0],
                  [0.0, sy]], dtype=np.float32)
    A = (R @ S).astype(np.float32)
    I = np.eye(2, dtype=np.float32)

    if not dragging:
        # Sincronizar t_state con trackbars SOLO si el usuario cambió Tx/Ty
        shown_tx = int(np.clip(round(t_state[0]) + CW//2, 0, CW))
        shown_ty = int(np.clip(round(t_state[1]) + CH//2, 0, CH))
        if (tx_ui + CW//2) != shown_tx or (ty_ui + CH//2) != shown_ty:
            t_state = np.array([tx_ui, ty_ui], dtype=np.float32)

    # Compensar cambio de pivote manteniendo b
    c_now = np.array([cx, cy], dtype=np.float32)
    if not dragging and not np.allclose(c_now, prev_c):
        b_keep = t_state + (I - A) @ prev_c
        t_state = b_keep - (I - A) @ c_now
        cv.setTrackbarPos("Tx", "Transformaciones", int(np.clip(round(t_state[0]) + CW//2, 0, CW)))
        cv.setTrackbarPos("Ty", "Transformaciones", int(np.clip(round(t_state[1]) + CH//2, 0, CH)))
        prev_c = c_now

    current_c = c_now.copy()
    A_cur = A
    I_cur = I

    b = (t_state + (I - A) @ c_now).astype(np.float32) # traslación final
    M = np.hstack([A, b.reshape(2,1)])
    M_cur = M

    out = cv.warpAffine(canvas0, M, (CW, CH))

    pc = (A @ c_now + b).astype(int) # pivote transformado p' = A c + b
    disp = out.copy()
    cv.circle(disp, (int(pc[0]), int(pc[1])), 6, (0,0,255), -1)
    cv.putText(disp, f"C=({int(cx)},{int(cy)})", (int(pc[0])+10, int(pc[1])-10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    view = disp[oy:oy+h, ox:ox+w]
    cv.imshow("Transformaciones", view)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()

Controles:
  • Clic izquierdo y arrastrar: trasladar imagen.
  • Clic derecho: fijar centro de rotación/escalado en el punto clicado.
  • Sliders: Tx/Ty/Cx/Cy/Ángulo/Escala. ESC para salir.


## Parte obligatoria sobre vídeo
###  1a. 
- Desarrollar una aplicación que lleve a cabo transformaciones del vídeo en tiempo real a través de una interfaz basada en trackbars o equivalente
- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.


In [ ]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("No se pudo abrir la cámara.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer frame de la cámara.")

h, w = frame.shape[:2]

# Lienzo 2x
CW, CH = w*2, h*2
ox, oy = (CW - w)//2, (CH - h)//2

cv.namedWindow("Transformaciones", cv.WINDOW_AUTOSIZE)
def nothing(_): pass

cv.createTrackbar("Tx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Ty", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Cy", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 300, nothing)  # 1.00 = 100
cv.createTrackbar("EscalaY", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("El punto (cx/cy) es el centro de rotación y de escalado. ESC para salir.")

# Transformación afín:
#     T(x) = A·(x - c) + c + t
# donde:
#     A -> rotación y escala combinadas
#     c -> centro de rotación/escalado
#     t -> traslación global
#
# OpenCV aplica:  x' = A·x + b
# donde:
#     b = t + (I - A)·c

t_state = np.array([0.0, 0.0], dtype=np.float32) # vector de traslación global (t_x, t_y) que desplaza toda la imagen
prev_c = np.array([                              # centro anterior de rotación/escalado (c_x, c_y), usado para compensar movimiento
    cv.getTrackbarPos("Cx", "Transformaciones"),
    cv.getTrackbarPos("Cy", "Transformaciones")
], dtype=np.float32)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    canvas0 = np.zeros((CH, CW, 3), np.uint8)
    canvas0[oy:oy+h, ox:ox+w] = frame  # imagen en el centro del lienzo

    tx_ui = cv.getTrackbarPos("Tx", "Transformaciones") - CW//2
    ty_ui = cv.getTrackbarPos("Ty", "Transformaciones") - CH//2
    ang = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    if cv.getTrackbarPos("Uniforme", "Transformaciones") == 1:
        sy = sx

    # Rotación y escalado
    rad = np.deg2rad(ang)
    c, s = np.cos(rad), np.sin(rad)
    R = np.array([[ c, -s],
                  [ s,  c]], dtype=np.float32)
    S = np.array([[sx, 0.0],
                  [0.0, sy]], dtype=np.float32)
    A = (R @ S).astype(np.float32)
    I = np.eye(2, dtype=np.float32)

    # Sincronizar t_state con trackbars SOLO si el usuario cambió Tx/Ty
    shown_tx = int(np.clip(round(t_state[0]) + CW//2, 0, CW))
    shown_ty = int(np.clip(round(t_state[1]) + CH//2, 0, CH))
    if (tx_ui + CW//2) != shown_tx or (ty_ui + CH//2) != shown_ty:
        t_state = np.array([tx_ui, ty_ui], dtype=np.float32)

    # Compensar cambio de pivote manteniendo b
    c_now = np.array([cx, cy], dtype=np.float32) # nuevo centro de rotación/escalado
    if not np.allclose(c_now, prev_c):
        b_keep = t_state + (I - A) @ prev_c # valor supuesto con centro anterior
        t_state = b_keep - (I - A) @ c_now # valor con nuevo centro
        cv.setTrackbarPos("Tx", "Transformaciones",
                          int(np.clip(round(t_state[0]) + CW//2, 0, CW)))
        cv.setTrackbarPos("Ty", "Transformaciones",
                          int(np.clip(round(t_state[1]) + CH//2, 0, CH)))
        prev_c = c_now

    b = (t_state + (I - A) @ c_now).astype(np.float32) # traslación final
    M = np.hstack([A, b.reshape(2,1)])  # 2x3
    out = cv.warpAffine(canvas0, M, (CW, CH))

    pc = (A @ c_now + b).astype(int) # pivote transformado p' = A c + b
    disp = out.copy()
    cv.circle(disp, (int(pc[0]), int(pc[1])), 6, (0,0,255), -1)
    cv.putText(disp, f"C=({cx},{cy})", (int(pc[0])+10, int(pc[1])-10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    view = disp[oy:oy+h, ox:ox+w]
    cv.imshow("Transformaciones", view)

    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()

El punto (cx/cy) es el centro de rotación y de escalado. ESC para salir.


### 1b. 
- Dado un vídeo trazar una ventana de proyección y proyectar el vídeo.

In [4]:
import cv2
import numpy as np

src_points = []
dst_points = []
phase = 0
frame_ref = None
homography_ready = False
H = None
dst_polygon = None

def ordenar_cuatro_puntos(pts):
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1) # x + y
    diff = np.diff(pts, axis=1) # y - x

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, frame_ref, H, homography_ready, dst_polygon

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 4:
            src_points.append([x, y])
            if len(src_points) == 4:
                print("Selecciona ahora los 4 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 4:
            dst_points.append([x, y])
            if len(dst_points) == 4:
                print("Calculando proyección...")
                src = ordenar_cuatro_puntos(src_points)
                dst = ordenar_cuatro_puntos(dst_points)
                H, _ = cv2.findHomography(src, dst)
                dst_polygon = np.int32(dst)
                homography_ready = True

def main():
    global frame_ref, H, homography_ready, dst_polygon

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("No se pudo abrir el video/cámara.")

    ret, frame_ref = cap.read()
    if not ret:
        raise RuntimeError("No se pudo leer el primer frame.")

    cv2.namedWindow("Proyeccion interactiva")
    cv2.setMouseCallback("Proyeccion interactiva", mouse_callback)

    print("Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output = frame.copy()

        if homography_ready and H is not None:
            h, w = frame.shape[:2]
            warped = cv2.warpPerspective(frame, H, (w, h))
            mask = np.zeros((h, w), dtype=np.uint8) # fondo negro
            cv2.fillPoly(mask, [dst_polygon], 255) # hueco blanco para la proyección
            result = np.zeros_like(frame) # crear lienzo vacío
            cv2.copyTo(warped, mask, result)

            output = result

        for (x, y) in src_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 255, 0), -1)
        for (x, y) in dst_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 0, 255), -1)

        cv2.imshow("Proyeccion interactiva", output)

        key = cv2.waitKey(30) & 0xFF
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).
Selecciona ahora los 4 puntos destino (rojo).
Calculando proyección...


### 1c. 
- Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello
los coeficientes de distorsión deben gobernarse a través de una interfaz

In [ ]:
import cv2
import numpy as np

cx, cy = None, None

def apply_distortion(image, k1, k2, p1, p2, k3, center=None):
    h, w = image.shape[:2]
    distCoeff = np.zeros((5,1), np.float64)
    distCoeff[0,0] = k1 # radial k1
    distCoeff[1,0] = k2 # radial k2
    distCoeff[2,0] = p1 # tangencial p1
    distCoeff[3,0] = p2 # tangencial p2
    distCoeff[4,0] = k3 # radial k3

    cam = np.eye(3, dtype=np.float32)
    if center is None:
        cam[0,2] = w / 2.0
        cam[1,2] = h / 2.0
    else:
        cam[0,2] = center[0]
        cam[1,2] = center[1]

    cam[0,0] = 10.0 # Definir longitud focal x
    cam[1,1] = 10.0 # Definir longitud focal y

    distorted_img = cv2.undistort(image, cam, distCoeff)
    return distorted_img

def mouse_click(event, x, y, flags, param):
    global cx, cy
    if event == cv2.EVENT_LBUTTONDOWN:
        cx, cy = x, y

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise FileNotFoundError("No se pudo abrir la cámara o el archivo de vídeo")

cv2.namedWindow("Distorsion interactiva (Video)")
cv2.setMouseCallback("Distorsion interactiva (Video)", mouse_click)

# Trackbars
cv2.createTrackbar("k1", "Distorsion interactiva (Video)", 50, 100, lambda x: None)
cv2.createTrackbar("k2", "Distorsion interactiva (Video)", 50, 100, lambda x: None)
cv2.createTrackbar("p1", "Distorsion interactiva (Video)", 50, 100, lambda x: None)
cv2.createTrackbar("p2", "Distorsion interactiva (Video)", 50, 100, lambda x: None)
cv2.createTrackbar("k3", "Distorsion interactiva (Video)", 50, 100, lambda x: None)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin del vídeo o error de lectura.")
        break

    k1 = (cv2.getTrackbarPos("k1", "Distorsion interactiva (Video)") - 50) / 5000.0
    k2 = (cv2.getTrackbarPos("k2", "Distorsion interactiva (Video)") - 50) / 5000.0
    p1 = (cv2.getTrackbarPos("p1", "Distorsion interactiva (Video)") - 50) / 5000.0
    p2 = (cv2.getTrackbarPos("p2", "Distorsion interactiva (Video)") - 50) / 5000.0
    k3 = (cv2.getTrackbarPos("k3", "Distorsion interactiva (Video)") - 50) / 5000.0

    distorted = apply_distortion(frame, k1, k2, p1, p2, k3, center=(cx, cy) if cx is not None else None)

    display = distorted.copy()
    if cx is not None and cy is not None:
        cv2.circle(display, (cx, cy), 5, (0, 0, 255), -1)

    cv2.imshow("Distorsion interactiva (Video)", display)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q") or key == 27:
        break

cap.release()
cv2.destroyAllWindows()


## Transformación afín con 3 puntos

In [5]:
import cv2
import numpy as np

src_points = []
dst_points = []
phase = 0
image = None
clone = None

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, image, clone

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 3:
            src_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Transformacion Afin", image)

            if len(src_points) == 3:
                print("Selecciona ahora los 3 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 3:
            dst_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow("Transformacion Afin", image)

            if len(dst_points) == 3:
                print("Calculando transformación afín...")
                aplicar_transformacion()

def aplicar_transformacion():
    global src_points, dst_points, clone, image

    src = np.array(src_points, dtype=np.float32)
    dst = np.array(dst_points, dtype=np.float32)

    M = cv2.getAffineTransform(src, dst)

    h, w = clone.shape[:2]
    warped = cv2.warpAffine(clone, M, (w, h))

    cv2.imshow("Transformacion Afin", warped)
    cv2.imwrite("output/resultado_affine.png", warped)
    print("Resultado guardado en 'resultado_affine.png'")

def main():
    global image, clone

    ruta = "images/cats.png"
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")
    image = clone.copy()

    cv2.namedWindow("Transformacion Afin")
    cv2.setMouseCallback("Transformacion Afin", mouse_callback)
    cv2.imshow("Transformacion Afin", image)

    print("Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).
Selecciona ahora los 3 puntos destino (rojo).
Calculando transformación afín...
Resultado guardado en 'resultado_affine.png'


## Calcular la imagen especular a partir de una imagen.

In [6]:
import cv2

ruta = "images/cats.png"
img = cv2.imread(ruta)
if img is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

espejo_horizontal = cv2.flip(img, 1)
espejo_vertical = cv2.flip(img, 0)
espejo_completo = cv2.flip(img, -1)

cv2.imshow("Original", img)
cv2.imshow("Espejo horizontal", espejo_horizontal)
cv2.imshow("Espejo vertical", espejo_vertical)
cv2.imshow("Espejo completo", espejo_completo)

cv2.imwrite("output/espejo_horizontal.png", espejo_horizontal)
cv2.imwrite("output/espejo_vertical.png", espejo_vertical)
cv2.imwrite("output/espejo_completo.png", espejo_completo)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Trazar una recta que será el eje de reflexión y “reflejar” la imagen.

In [ ]:
import cv2
import numpy as np
import os

points = []
clone = None
done = False

os.makedirs("output", exist_ok=True)

def mouse_callback(event, x, y, flags, param):
    global points, clone, done

    if done:
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append([x, y])
        if len(points) == 2:
            reflected_clean = warp_reflection(clone, points[0], points[1])
            cv2.imwrite("output/resultado_reflexion.png", reflected_clean)

            reflected_with_line = update_reflection(points[1])
            cv2.imwrite("output/resultado_reflexion_con_linea.png", reflected_with_line)

            print("Guardadas imágenes en output/")
            done = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if len(points) == 1:
            display = update_reflection([x, y])
            cv2.imshow("Reflexion", display)

def update_reflection(temp_point=None):
    global clone, points

    if len(points) == 0:
        return clone.copy()
    elif len(points) == 1: # vista en tiempo real
        p1 = np.array(points[0], dtype=np.float32)
        if temp_point is None:
            return clone.copy()
        p2 = np.array(temp_point, dtype=np.float32)
    else: # vista definitiva
        p1 = np.array(points[0], dtype=np.float32)
        p2 = np.array(points[1], dtype=np.float32)

    h, w = clone.shape[:2]
    M_affine = get_affine_matrix(p1, p2)
    reflected = cv2.warpAffine(clone, M_affine, (w, h))

    display_reflected = reflected.copy()
    cv2.line(display_reflected, tuple(p1.astype(int)), tuple(p2.astype(int)), (255, 0, 0), 2)
    cv2.circle(display_reflected, tuple(p1.astype(int)), 5, (0, 0, 255), -1)
    cv2.circle(display_reflected, tuple(p2.astype(int)), 5, (0, 0, 255), -1)

    return display_reflected

def get_affine_matrix(p1, p2):
    dx, dy = p2 - p1
    ang = np.arctan2(dy, dx)

    T1 = np.array([[1, 0, -p1[0]],
                   [0, 1, -p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    R = np.array([[ np.cos(-ang), -np.sin(-ang), 0],
                  [ np.sin(-ang),  np.cos(-ang), 0],
                  [0, 0, 1]], dtype=np.float32)
    Ref = np.array([[1, 0, 0],
                    [0, -1, 0],
                    [0, 0, 1]], dtype=np.float32)
    R_inv = np.linalg.inv(R)
    T2 = np.array([[1, 0, p1[0]],
                   [0, 1, p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    M = T2 @ R_inv @ Ref @ R @ T1
    return M[:2, :]

def warp_reflection(img, p1, p2):
    h, w = img.shape[:2]
    M_affine = get_affine_matrix(np.array(p1, dtype=np.float32), np.array(p2, dtype=np.float32))
    return cv2.warpAffine(img, M_affine, (w, h))

def main():
    global clone
    ruta = "images/Gran_Canaria.jpg"
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")

    cv2.namedWindow("Original")
    cv2.namedWindow("Reflexion")

    cv2.imshow("Original", clone)

    cv2.setMouseCallback("Reflexion", mouse_callback)
    cv2.imshow("Reflexion", clone)

    print("Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.
Guardadas imágenes en output/
